We wish to form the interpolant

where $\vec{y} = \vec{x}_1$ (for notational convenience) and $\mathcal{L}_\vec{x} = \vec{n}\cdot \nabla_\vec{x}$

In [14]:
%matplotlib notebook
import numpy as np
import scipy.linalg as la
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree

import scipy.sparse as sp
from scipy.sparse.linalg import spsolve, lsqr

from numpy.linalg import cond, norm, inv, lstsq

from poly_basis import *
from spherepts import *
from rbf import *

from math import *

In [3]:
import matplotlib
matplotlib.rcParams.update({'font.size': 14})

In [40]:
rbf_obj = rbf_dict['multiquadric']
eps = 1
rbf = lambda r: rbf_obj['rbf'](r, eps)
zeta = lambda r: rbf_obj['zeta'](r, eps)
chi = lambda r: rbf_obj['chi'](r, eps)
beta = lambda r: rbf_obj['beta'](r, eps)

In [41]:
zeta(1)

0.7071067811865475

In [82]:
x = np.array([.1, .2, .3])
y = np.array([.2,  0, .4])
n = np.array([.1, .2, .3])
n = n/la.norm(n)

In [95]:
def Hess(x, y, n):
    r = sqrt(sum([(a-b)**2 for a,b in zip(x,y)]))
    H = [[zeta(r)+(x[0]-y[0])**2 * chi(r), (x[0]-y[0])*(x[1]-y[1])*chi(r) , (x[0]-y[0])*(x[2]-y[2])*chi(r) ],
         [(x[0]-y[0])*(x[1]-y[1])*chi(r) , zeta(r)+(x[1]-y[1])**2 * chi(r), (x[1]-y[1])*(x[2]-y[2])*chi(r) ],
         [(x[0]-y[0])*(x[2]-y[2])*chi(r) , (x[1]-y[1])*(x[2]-y[2])*chi(r) , zeta(r)+(x[2]-y[2])**2 * chi(r)]]
    H = np.array(H)    
    return n.T @ H @ n

def L2(x,y,n):
    r = sqrt(sum([(a-b)**2 for a,b in zip(x,y)]))
    #return n @ (n*zeta(r) + x**2*chi(r)) - n@n + (n@x)*chi(r)
    #return n@n + (n@x)*chi(r)
    return zeta(r) + (n @ (x-y))**2*chi(r)

In [96]:
H = Hess(x,y, n)
L = L2(x,y,n)
print(H)
print(L)

0.9712858623572641
0.9712858623572642


In [62]:
H

array([[ 0.96212279, -0.01832615, -0.02748922],
       [-0.01832615,  0.93463357, -0.05497845],
       [-0.02748922, -0.05497845,  0.88881819]])

In [64]:
n.T @ H

array([0.08430028, 0.16860056, 0.25290085])

In [65]:
H @ n

array([0.08430028, 0.16860056, 0.25290085])

In [66]:
n @ x

0.14